In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[1]").appName("sid_spark").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/19 22:45:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
rdd = spark.sparkContext.parallelize([1,2,3,4,5])

In [3]:
rdd.count()

5